# Q2_radiation_accessibility

In [1]:
import requests
import json
import pandas as pd
import geopandas as gpd
from pandas.core.frame import DataFrame
import os
import statistics as st
from osgeo import ogr
from shapely.geometry import MultiPolygon
from datetime import datetime
import pytz

In [2]:
#origin point csv
office=pd.read_csv('data/building.csv')
factory=pd.read_csv('data/industry.csv')
farm=pd.read_csv('data/farm.csv')
resident=gpd.read_file('data/centroids.shp')

#destination point csv
hospital=pd.read_csv('data/hospital.csv')
park=pd.read_csv('data/park.csv')
shop=pd.read_csv('data/shop.csv')

#supporting documents
township_list=pd.read_csv('data/Town_district_list.csv')

## Processing the Centroids shp to csv

In [3]:
resident=resident.to_crs("EPSG:4326")

In [4]:
resident.to_csv('data/resident.csv')

In [5]:
resident=pd.read_csv('data/resident.csv')

In [6]:
resident

,Unnamed: 0,OBJECTID,Name,Layer,gml_id,code,grade,Shape_Leng,Shape_Area,geometry
0,0,1,沙河街道,乡镇,NaN,NaN,0,20040.978853,2.179247e+07,POINT (113.9717641239288 22.53587233948723)
1,1,2,粤海街道,乡镇,NaN,NaN,0,19159.589736,1.807562e+07,POINT (113.9392492352695 22.5302223288828)
2,2,13,凤凰街道,乡镇,layer_township_pg.15697,4.403110e+11,4,29736.850664,2.598523e+07,POINT (113.9302728066271 22.73682204640801)
3,3,14,玉塘街道,乡镇,layer_township_pg.15698,4.403110e+11,4,29501.182969,2.532503e+07,POINT (113.8961826861495 22.71762848830217)
4,4,15,光明街道,乡镇,layer_township_pg.15699,4.403110e+11,4,30030.513705,3.424578e+07,POINT (113.9618129279743 22.75200408670002)
...,...,...,...,...,...,...,...,...,...,...
69,69,88,大鹏街道,乡镇,layer_township_pg.15773,4.403070e+11,4,63412.662042,8.879811e+07,POINT (114.5036482850048 22.60199187379314)
70,70,19,南山街道,乡镇,layer_township_pg.15704,4.403050e+11,4,33035.172793,3.875446e+07,POINT (113.8911273584019 22.51356938009944)
71,71,48,海山街道,乡镇,layer_township_pg.15733,4.403080e+11,4,12242.843502,8.851195e+06,POINT (114.2298682927088 22.56426415049962)
72,72,43,盐田街道,乡镇,layer_township_pg.15728,4.403080e+11,4,32169.232404,5.358986e+07,POINT (114.2610436796775 22.60266935201232)


In [7]:
resident[['point','cood']]=resident['geometry'].str.split('(', expand=True)
resident[['lng','lat']]=resident['cood'].str.split(' ', expand=True)
resident[['lat','no']]=resident['lat'].str.split(')', expand=True)
resident = resident[['Name','lat','lng']].copy()
resident=resident.rename(columns = {'Name':'name'})
resident

,name,lat,lng
0,沙河街道,22.53587233948723,113.9717641239288
1,粤海街道,22.5302223288828,113.9392492352695
2,凤凰街道,22.73682204640801,113.9302728066271
3,玉塘街道,22.71762848830217,113.8961826861495
4,光明街道,22.75200408670002,113.9618129279743
...,...,...,...
69,大鹏街道,22.60199187379314,114.5036482850048
70,南山街道,22.51356938009944,113.8911273584019
71,海山街道,22.56426415049962,114.2298682927088
72,盐田街道,22.60266935201232,114.2610436796775


In [8]:
resident_new=resident.merge(township_list, how='inner', on='name')
resident_new['lat']=resident_new['lat'].apply(lambda x: float(x)).round(6)
resident_new['lng']=resident_new['lng'].apply(lambda x: float(x)).round(6)
resident_new

,name,lat,lng,district,house_price_level
0,沙河街道,22.535872,113.971764,南山,high
1,粤海街道,22.530222,113.939249,南山,high
2,凤凰街道,22.736822,113.930273,光明,medium
3,玉塘街道,22.717628,113.896183,光明,medium
4,光明街道,22.752004,113.961813,光明,medium
...,...,...,...,...,...
69,大鹏街道,22.601992,114.503648,大鹏,low
70,南山街道,22.513569,113.891127,南山,high
71,海山街道,22.564264,114.229868,盐田,medium
72,盐田街道,22.602669,114.261044,盐田,medium


In [9]:
resident_high=resident_new[resident_new['house_price_level']=='high']
resident_medium=resident_new[resident_new['house_price_level']=='medium']
resident_low=resident_new[resident_new['house_price_level']=='low']

In [10]:
shop

,Unnamed: 0,name,lat,lng,district
0,0,华润万家(龙华店),22.660378,114.027651,龙华区
1,1,华润万家(益田店),22.529075,114.063573,福田区
2,2,华润万家(沙井店),22.734317,113.834571,宝安区
3,3,华润万家(横岗店),22.643722,114.198062,龙岗区
4,4,华润万家(宝安店),22.566492,113.911443,宝安区
...,...,...,...,...,...
76,82,华润万家龙岗配送中心,22.702591,114.320910,龙岗区
77,83,华润万家便利超市(翠枫店),22.617799,114.108880,龙岗区
78,84,华润万家便利超市(支六路店),22.495580,113.948922,南山区
79,85,华润万家(麓园店),22.738238,114.226041,龙岗区


## Defining the functions

In [11]:
#Input: start latitude, start longitude, end latitude, end longitude
def stod(slat,slng,dlat,dlng):
    url ="http://api.map.baidu.com/direction/v2/transit?" #API address
    ak = 'cGCdY6KHFma1t35LhDUZ13WI5qG1ni24' #API key
    tac_metro = r'&tactics_incity=5' #Route planning with metro as the priority mode of travel
    real_url = url +"origin="+slat+","+slng+"&destination="+dlat+","+dlng+tac_metro+"&ak="+ak 
    req = requests.get(real_url)
    t = req.text
    data = json.loads(t) #save the data in the array data
    try: #Prevent some data errors from causing the request to terminate
        total_duration = data['result']["routes"][0]["duration"]/60
        #Get the total travel time of this piece of data
        stepstr = "Total time:"+str(total_duration) 
        #Get all steps of this roate
        steps = data['result']["routes"][0]['steps'] 
        for step in steps:
            # Introduction of each step including its mode of transportation
            step_tructions = step[0]["instructions"]
            # time spent in each step
            step_duration = step[0]["duration"]
            # Set a delimiter for the result for subsequent processing
            stepstr = stepstr+"/"+step_tructions+":"+str(step_duration/60)
    except:
        stepstr = None
    req.close()
    # The return data is the total duration/first step/first step time/second step/second step time/...
    return stepstr

In [12]:
# Sets the function that computes the average of the list for later use.
def ave(lst):
    return sum(lst) / len(lst)

In [13]:
# Set the function to get the name of the dataframe for later file saving.
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [14]:
# Set up a function sor automatic loops, calculations and exported

def stodabc(csv_o,csv_d):
    
    # Get the local time and date when data collection started
    start_time = datetime.now(pytz.timezone('Asia/Shanghai'))
    st_string = start_time.strftime("%d/%m/%Y %H:%M:%S")
    
    #Set some empty lists for later use
    only_bus=[]
    only_walk=[]
    wrong_data=[]
    metro=[]
    walk=[]
    other=[]
    traffic=[]
    waiting=[]
    total=[]
    transfer=[]
    
    metro_min=[]
    metro_traffic_min=[]
    metro_transfer_min=[]
    
    traffic_min=[]
    traffic_metro_min=[]
    traffic_transfer_min=[]
    
    metro_median=[]
    traffic_median=[]
    transfer_median=[]

    # For all "starting data" in csv
    for o in range (len(csv_o['lat'])):

        d_metro_time_total=[]
        d_walk_time_total=[]
        d_other_time_total=[]
        d_total_time=[]
        d_waiting_time=[]
        d_status=[]
        d_transfer=[]
        d_traffic_time=[]

        # For all endpoint data in csv
        for d in range(len(csv_d['lat'])):
            s=str(stod(str(csv_o['lat'][o]),str(csv_o['lng'][o]),str(csv_d['lat'][d]),str(csv_d['lng'][d])))

            # From a starting point to each destination, if there is no line to reach the destination by metro:
            if s.find('乘地铁')==-1:
                
                # #Only accessible by bus
                if s.find('乘')!=-1:
                    result='Only bus can arrive'
                    i_walk_time_total=-10000
                    i_other_time_total=10000

                # #If it can only be reached by foot
                elif s.find('乘')==-1 and s.find('步行')!=-1:
                    result='Within walking distance'
                    i_walk_time_total=10000
                    i_other_time_total=-10000

                # #Unreachable case
                else:
                    result='Wrong cause without walking data, bus data or metro data'
                    i_walk_time_total=0
                    i_other_time_total=0
                i_metro_time_total=0
                i_total_time=0
                i_waiting_time=0
                i_transfer=-10000
                i_traffic_time=0

            # For the case where there is a route to the destination by metro
            elif s.find('乘地铁')!=-1:
                result='Can be arrived by metro'
                ## Calculate the number of transfers
                i_transfer=s.count('站内换乘')
                
                ##Process the data from the starting point to one of the ending points into a table
                first_split = s.split("/")
                dic= {'step':first_split}
                table = DataFrame(dic)
                table[['step','time']] = table['step'].str.split(':', expand=True)
                # table['time'] = table['time'].fillna(0)

                # # Extract data from the above table
                i_metro_time=[]
                i_walk_time=[]
                i_other_time=[]
                for i in range(len(table)):
                    # Extract metro ride time
                    if table['step'][i].find('乘地铁')!=-1 or table['step'][i].find('站内换乘')!=-1:
                        i_metro_time.append(float(table['time'][i]))
                    # Extract walking time
                    elif table['step'][i].find('乘地铁')==-1 and table['step'][i].find('步行')!=-1 and table['step'][i].find('站内换乘')==-1:
                        i_walk_time.append(float(table['time'][i]))
                    # Extract the whole time (including waiting time)
                    elif table['step'][i].find('Total time')!=-1:
                        i_total_time=float(table['time'][i])
                    # Extract the time of other means of transportation
                    else:
                        i_other_time.append(float(table['time'][i]))

                # Calculate the total time of each type of time
                i_metro_time_total=sum(i_metro_time)
                i_walk_time_total=sum(i_walk_time)
                i_other_time_total=sum(i_other_time)
                # #Extract time spent using transportation and walking
                i_traffic_time=i_metro_time_total+i_walk_time_total+i_other_time_total
                #Calculate the waiting time
                i_waiting_time=i_total_time-i_traffic_time

            # # Handling other error data
            else:
                result='Wrong cause contradict result'
                i_metro_time_total=404
                i_walk_time_total=404
                i_other_time_total=404
                i_total_time=404
                i_waiting_time=404
                i_transfer=404
                i_traffic_time=404

            # #Package all the above data from the starting point to each end point
            d_metro_time_total.append(i_metro_time_total)
            d_walk_time_total.append(i_walk_time_total)
            d_other_time_total.append(i_other_time_total)
            d_total_time.append(i_total_time)
            d_waiting_time.append(i_waiting_time)
            d_status.append(result)
            d_transfer.append(i_transfer)
            d_traffic_time.append(i_traffic_time)

        d_dic={'Destination':csv_d['name'],
               'Status':d_status,
               'Metro_time':d_metro_time_total,
               'Walk_time':d_walk_time_total,
               'Other_time':d_other_time_total,
               'Traffic_time':d_traffic_time,
               'Waiting_time':d_waiting_time,
               'Total_time':d_total_time,
               'Transfer':d_transfer}
        d_final=DataFrame(d_dic)

        # #Consider the processing of different result data
        f_only_bus=[]
        f_only_walk=[]
        f_wrong_data=[]
        f_metro=[]
        f_walk=[]
        f_other=[]
        f_traffic=[]
        f_waiting=[]
        f_total=[]
        f_transfer=[]
        for f in range (len(d_final['Destination'])):
            if d_final['Status'][f]=='Only bus can arrive':
                f_only_bus.append(d_final['Other_time'][f])
            elif d_final['Status'][f]=='Within walking distance':
                f_only_walk.append(d_final['Walk_time'][f])
            elif d_final['Status'][f]=='Can be arrived by metro':
                f_metro.append(d_final['Metro_time'][f])
                f_walk.append(d_final['Walk_time'][f])
                f_other.append(d_final['Other_time'][f])
                f_traffic.append(d_final['Traffic_time'][f])
                f_waiting.append(d_final['Waiting_time'][f])
                f_total.append(d_final['Total_time'][f])
                f_transfer.append(d_final['Transfer'][f])
            else:
                f_wrong_data.append(d_final['Transfer'][f])

        # Calculate the average from one starting point to all ending points, 
        # and pack it into a list of all starting point results
        only_bus.append(len(f_only_bus))
        only_walk.append(len(f_only_walk))
        wrong_data.append(len(f_wrong_data))
        metro.append(ave(f_metro))
        walk.append(ave(f_walk))
        other.append(ave(f_other))
        traffic.append(ave(f_traffic))
        waiting.append(ave(f_waiting))
        total.append(ave(f_total))
        transfer.append(ave(f_transfer))
        
        # Calculate the shortest metro rade time from one starting point to all ending points, etc.,
        # and pack it into a list of all starting point results
        metro_min.append(min(f_metro))
        metro_traffic_min.append(f_traffic[f_metro.index(min(f_metro))])
        metro_transfer_min.append(f_transfer[f_metro.index(min(f_metro))])
        
        # Calculate the shortest traffic time from one starting point to all ending points, etc.,
        # and pack it into a list of all starting point results
        traffic_min.append(min(f_traffic))
        traffic_metro_min.append(f_metro[f_traffic.index(min(f_traffic))])
        traffic_transfer_min.append(f_transfer[f_traffic.index(min(f_traffic))])
        
        # Calculate the median from one starting point to all ending points, 
        # and pack it into a list of all starting point results
        metro_median.append(st.median(f_metro))
        traffic_median.append(st.median(f_traffic))
        transfer_median.append(st.median(f_transfer))
        
        #Print recent data to visit progress as it runs
        print(str(only_bus)+","+str(only_walk)+","+str(wrong_data)+","+str(metro)+","+str(walk)+","+str(other)+","+str(traffic)+","+str(waiting)+","+str(total)+","+str(transfer)+
             str(metro_min)+","+str(metro_traffic_min)+","+str(metro_transfer_min)+","+
             str(traffic_min)+","+str(traffic_metro_min)+","+str(traffic_transfer_min)+","+
             str(metro_median)+","+str(traffic_median)+","+str(transfer_median))

    # Make the result into a dataframe
    dic={'Origin':csv_o['name'],
         'Average_metro_time':metro,
         'Average_walk_time':walk,
         'Average_other_time':other,
         'Average_traffic_time':traffic,
         'Average_waiting_time':waiting,
         'Average_total_time':total,
         'Average_metro_line_transfer':transfer,
         'Min_metro_time':metro_min,
         'Traffic_time_of min_metro_time':metro_traffic_min,
         'Transfer_of_min_metro_time':metro_transfer_min,
         'Min_traffic_time':traffic_min,
         'Metro_time_of_min_traffic_time':traffic_metro_min,
         'Transfer_of_min_traffic_time':traffic_transfer_min,
         'Median_metro_time':metro_median,
         'Median_traffic_time':traffic_median,
         'Median_metro_line_transfer':transfer_median,
        'Count_of_only_bus_round':only_bus,
        'Count_of only_walk_round':only_walk,
        'Count_of_none_data':wrong_data}
    
    final=DataFrame(dic)
    print(final)
    
    # Get the local time and date when data collection ended
    end_time = datetime.now(pytz.timezone('Asia/Shanghai'))
    et_string = end_time.strftime("%d/%m/%Y %H:%M:%S")
    
    # Print started and ended time
    print("The start time of collection in China is : ")
    print(st_string)
    print('The end time of collection in China is : ')
    print(et_string)
    
    # save dataframe to folder as a csv
    final.to_csv('output_data/'+str( get_df_name(csv_o))+'_to_'+str( get_df_name(csv_d))+'.csv')

In [115]:
#Because the daily quota limitation of API, results cannot be obtained in one time. 
#The following code have been abandoned.
'''
origin=[farm,factory,office,resident_low,resident_high,resident_medium]
destination=[hospital,shop,park]

for o_file in origin:
    for d_file in destination:
        stodabc(o_file,d_file)
'''


'\norigin=[farm,factory,office]\ndestination=[hospital,shop,park]\n\nfor o_file in origin:\n    for d_file in destination:\n        stodabc(o_file,d_file)\n'

In [ ]:
stodabc(farm,shop)

In [15]:
stodabc(resident_high,shop)

[0],[0],[0],[40.56049382716049],[25.58765432098766],[7.512345679012344],[73.6604938271605],[7.901234567901234],[81.5617283950617],[0.5802469135802469][3.0],[56.15],[0],[20.366666666666667],[5.0],[0],[41.0],[66.03333333333333],[1]
[0, 0],[0, 0],[0, 1],[40.56049382716049, 43.37583333333335],[25.58765432098766, 21.938749999999995],[7.512345679012344, 8.325],[73.6604938271605, 73.6395833333333],[7.901234567901234, 8.906250000000002],[81.5617283950617, 82.5458333333333],[0.5802469135802469, 1.0375][3.0, 3.0],[56.15, 51.483333333333334],[0, 0],[20.366666666666667, 21.45],[5.0, 4.0],[0, 0],[41.0, 42.0],[66.03333333333333, 67.67500000000001],[1, 1.0]


KeyError: 2

# __________________________________________ abandoned code________________________________________

In [22]:
#origin point csv
factory_part=pd.read_csv('data/industry_part.csv')
farm_part=pd.read_csv('data/farm_part.csv')

#destination point csv
hospital_part=pd.read_csv('data/hospital_part.csv')
shop_part=pd.read_csv('data/shop_part.csv')

In [23]:
stodabc(farm,shop_part)

[0],[0],[0],[73.56666666666666],[19.416666666666668],[7.177777777777777],[100.16111111111111],[15.0],[115.16111111111111],[1.0][55.68333333333334],[82.7],[2],[82.7],[55.68333333333334],[2],[57.0],[85.35],[1]
[0, 0],[0, 0],[0, 0],[73.56666666666666, 38.550000000000004],[19.416666666666668, 13.449999999999998],[7.177777777777777, 0.0],[100.16111111111111, 52.0],[15.0, 5.000000000000003],[115.16111111111111, 57.0],[1.0, 1.0][55.68333333333334, 26.116666666666667],[82.7, 37.55],[2, 1],[82.7, 37.55],[55.68333333333334, 26.116666666666667],[2, 1],[57.0, 43.96666666666667],[85.35, 59.03333333333333],[1, 1]
[0, 0, 0],[0, 0, 0],[0, 0, 0],[73.56666666666666, 38.550000000000004, 37.855555555555554],[19.416666666666668, 13.449999999999998, 12.883333333333333],[7.177777777777777, 0.0, 0.0],[100.16111111111111, 52.0, 50.73888888888889],[15.0, 5.000000000000003, 5.0],[115.16111111111111, 57.0, 55.73888888888889],[1.0, 1.0, 1.0][55.68333333333334, 26.116666666666667, 28.066666666666666],[82.7, 37.55, 

In [24]:
factory_part

,name,lat,lng,district
0,启迪协信(深圳)科技园,22.703539,114.215713,龙岗区
1,中国科技开发院中科研发园,22.544152,113.959029,南山区
2,创智云城,22.585013,113.946114,南山区


In [25]:
shop_part

,Unnamed: 0,name,lat,lng,district
0,0,华润万家(龙华店),22.660378,114.027651,龙华区
1,1,华润万家(益田店),22.529075,114.063573,福田区
2,2,华润万家(沙井店),22.734317,113.834571,宝安区


In [28]:
stod('22.703539','114.215713','22.660378','114.027651')

'Total time:100.2/步行307米:4.383333333333334/启迪协信科技园站乘M593路(龙岗信息学院总站(不下客)方向)经过4站到龙城水厂站:6.566666666666666/步行706米:10.016666666666667/大运站(C口)乘地铁3号线(福保方向)经过9站到布吉站:24.0/站内换乘:0.6166666666666667/布吉站乘地铁5号线(赤湾方向)经过8站到深圳北站:19.0/站内换乘 步行77米:1.0666666666666667/深圳北站乘地铁4号线(牛湖方向)经过4站到龙华站(C口):11.0/步行427米:6.05'

In [73]:
s = '155.08333333333334/步行1.4公里,20.3/大鹏所城站乘大鹏假日专线6号(盐田路地铁站方向)经过9站到盐田路地铁站,42.13333333333333/步行136米,1.8833333333333333/盐田路站(G口)乘地铁2号线(赤湾方向)经过16站到市民中心站,43.0/站内换乘 步行196米,2.716666666666667/市民中心站乘地铁4号线(牛湖方向)经过10站到龙华站(C口),24.0/步行427米,6.05'
if s.find('乘地铁')==-1:
    if s.find('乘')!=-1:
        result='Only bus can arrive'
    elif s.find('乘')==-1 and s.find('步行')!=-1:
        result='Within walking distance'
    else:
        result='Wrong cause without walking data, bus data or metro data'
        
elif s.find('乘地铁')!=-1:
    result='Can be arrived by metro'
    transfer=s.count('换乘')
    first_split = s.split("/")
    dic= {'step':first_split}
    table = DataFrame(dic)
    table[['step','time']] = table['step'].str.split(',', expand=True)
    table['time'] = table['time'].fillna(0)
    
    i_metro_time=[]
    i_walk_time=[]
    i_other_time=[]
    for i in range(len(table)):
        if table['step'][i].find('乘地铁')!=-1 or table['step'][i].find('换乘')!=-1:
            i_metro_time.append(float(table['time'][i]))
        elif table['step'][i].find('乘地铁')==-1 and table['step'][i].find('步行')!=-1:
            i_walk_time.append(float(table['time'][i]))
        else:
            i_other_time.append(float(table['time'][i]))
    
    i_metro_time_total=sum(i_metro_time)
    i_walk_time_total=sum(i_walk_time)
    i_other_time_total=sum(i_other_time)
    total_time=i_metro_time_total+i_walk_time_total+i_other_time_total
    waiting_time=float(table['step'][0])-total_time
    
else:
    result='Wrong cause contradict result'

print(result)
print(total_time)
print(i_metro_time_total)
print(i_walk_time_total)
print(i_other_time_total)
print(waiting_time)
table

Can be arrived by metro
140.08333333333334
69.71666666666667
28.233333333333334
42.13333333333333
15.0


,step,time
0,155.08333333333334,0
1,步行1.4公里,20.3
2,大鹏所城站乘大鹏假日专线6号(盐田路地铁站方向)经过9站到盐田路地铁站,42.13333333333333
3,步行136米,1.8833333333333333
4,盐田路站(G口)乘地铁2号线(赤湾方向)经过16站到市民中心站,43.0
5,站内换乘 步行196米,2.716666666666667
6,市民中心站乘地铁4号线(牛湖方向)经过10站到龙华站(C口),24.0
7,步行427米,6.05


In [69]:
    o_only_bus_count=len(f_only_bus)
    o_only_walk_count=len(f_only_walk)
    o_wrong_data_count=len(f_wrong_data)
    o_ave_metro=ave(f_metro)
    o_ave_walk=ave(f_walk)
    o_ave_other=ave(f_other)
    o_ave_traffic=ave(f_traffic)
    o_ave_waiting=ave(f_waiting)
    o_ave_total=ave(f_total)
    o_ave_transfer=ave(f_transfer)

,step,time
0,155.08333333333334,0
1,步行1.4公里,20.3
2,大鹏所城站乘大鹏假日专线6号(盐田路地铁站方向)经过9站到盐田路地铁站,42.13333333333333
3,步行136米,1.8833333333333333
4,盐田路站(G口)乘地铁2号线(赤湾方向)经过16站到市民中心站,43.0
5,站内换乘 步行196米,2.716666666666667
6,市民中心站乘地铁4号线(牛湖方向)经过10站到龙华站(C口),24.0
7,步行427米,6.05


In [109]:
table

,step,time
0,"Total time,176.55",None
1,步行759米,10.833333333333334
2,榕树坑站乘M471路(大鹏总站方向)或M457路经过12站到大鹏中心①站,20.566666666666666
3,同站换乘,0.08333333333333333
4,大鹏中心①站乘E11路(深圳北站交通枢纽公交场站方向)经过7站到银湖汽车站②站,82.11666666666666
5,步行164米,2.2666666666666666
6,银湖站(D口)乘地铁9号线(前湾方向)经过2站到上梅林站,6.0
7,站内换乘 步行334米,4.633333333333334
8,上梅林站乘地铁4号线(牛湖方向)经过7站到龙华站(C口),19.0
9,步行427米,6.05


In [1]:
'''
for o in range (len(farm['lat'])):
    for d in range(len(shop['lat'])):
        print(stod(str(farm['lat'][o]),str(farm['lng'][o]),str(shop['lat'][d]),str(shop['lng'][d])))
        '''

"\nfor o in range (len(farm['lat'])):\n    for d in range(len(shop['lat'])):\n        print(stod(str(farm['lat'][o]),str(farm['lng'][o]),str(shop['lat'][d]),str(shop['lng'][d])))\n        "

In [2]:
'''
for o in range (len(farm['lat'])):
    
    for d in range(len(shop['lat'])):
        s=str(stod(str(farm['lat'][o]),str(farm['lng'][o]),str(shop['lat'][d]),str(shop['lng'][d])))
        
        if s.find('乘地铁')==-1:
            if s.find('乘')!=-1:
                result='Only bus can arrive'
                table=1
                
            elif s.find('乘')==-1 and s.find('步行')!=-1:
                result='Within walking distance'
                table=2
                
            else:
                result='Wrong cause without walking data, bus data or metro data'
                table=3
        
        elif s.find('乘地铁')!=-1:
            result='Can be arrived by metro'
            first_split = s.split("/")
            dic= {'step':first_split}
            table = DataFrame(dic)
            table[['step','time']] = table['step'].str.split(':', expand=True)
    
        else:
            result='Wrong cause contradict result'
            table=404
        print(result)
        table
        '''

'\nfor o in range (len(farm[\'lat\'])):\n    \n    for d in range(len(shop[\'lat\'])):\n        s=str(stod(str(farm[\'lat\'][o]),str(farm[\'lng\'][o]),str(shop[\'lat\'][d]),str(shop[\'lng\'][d])))\n        \n        if s.find(\'乘地铁\')==-1:\n            if s.find(\'乘\')!=-1:\n                result=\'Only bus can arrive\'\n                table=1\n                \n            elif s.find(\'乘\')==-1 and s.find(\'步行\')!=-1:\n                result=\'Within walking distance\'\n                table=2\n                \n            else:\n                result=\'Wrong cause without walking data, bus data or metro data\'\n                table=3\n        \n        elif s.find(\'乘地铁\')!=-1:\n            result=\'Can be arrived by metro\'\n            first_split = s.split("/")\n            dic= {\'step\':first_split}\n            table = DataFrame(dic)\n            table[[\'step\',\'time\']] = table[\'step\'].str.split(\':\', expand=True)\n    \n        else:\n            result=\'Wrong cau

In [3]:
'''
for o in range (len(farm['lat'])):
    
    d_metro_time_total=[]
    d_walk_time_total=[]
    d_other_time_total=[]
    d_total_time=[]
    d_waiting_time=[]
    d_status=[]
    d_transfer=[]
    d_traffic_time=[]
    
    for d in range(len(shop['lat'])):
        s=stod(str(farm['lat'][o]),str(farm['lng'][o]),str(shop['lat'][d]),str(shop['lng'][d]))
        
        if s.find('乘地铁')==-1:
            if s.find('乘')!=-1:
                result='Only bus can arrive'
                i_walk_time_total=-10000
                i_other_time_total=10000
                
            elif s.find('乘')==-1 and s.find('步行')!=-1:
                result='Within walking distance'
                i_walk_time_total=10000
                i_other_time_total=-10000
                
            else:
                result='Wrong cause without walking data, bus data or metro data'
                i_walk_time_total=0
                i_other_time_total=0
            i_metro_time_total=0
            i_total_time=0
            i_waiting_time=0
            i_transfer=-10000
            i_traffic_time=0
        
        elif s.find('乘地铁')!=-1:
            result='Can be arrived by metro'
            i_transfer=s.count('站内换乘')
            first_split = s.split("/")
            dic= {'step':first_split}
            table = DataFrame(dic)
            table[['step','time']] = table['step'].str.split(',', expand=True)
            # table['time'] = table['time'].fillna(0)
    
            i_metro_time=[]
            i_walk_time=[]
            i_other_time=[]
            for i in range(len(table)):
                if table['step'][i].find('乘地铁')!=-1 or table['step'][i].find('换乘')!=-1:
                    i_metro_time.append(float(table['time'][i]))
                elif table['step'][i].find('乘地铁')==-1 and table['step'][i].find('步行')!=-1:
                    i_walk_time.append(float(table['time'][i]))
                elif table['step'][i].find('Total time')!=-1:
                    i_total_time=float(table['time'][i])
                else:
                    i_other_time.append(float(table['time'][i]))
    
            i_metro_time_total=sum(i_metro_time)
            i_walk_time_total=sum(i_walk_time)
            i_other_time_total=sum(i_other_time)
            i_traffic_time=i_metro_time_total+i_walk_time_total+i_other_time_total
            i_waiting_time=i_total_time-i_traffic_time
    
        else:
            result='Wrong cause contradict result'
            i_metro_time_total=404
            i_walk_time_total=404
            i_other_time_total=404
            i_total_time=404
            i_waiting_time=404
            i_transfer=404
            i_traffic_time=404
        
        d_metro_time_total.append(i_metro_time_total)
        d_walk_time_total.append(i_walk_time_total)
        d_other_time_total.append(i_other_time_total)
        d_total_time.append(i_total_time)
        d_waiting_time.append(i_waiting_time)
        d_status.append(result)
        d_transfer.append(i_transfer)
        d_traffic_time.append(i_traffic_time)
        
    d_dic={'Destination':shop['name'],
           'Status':d_status,
           'Metro_time':d_metro_time_total,
           'Walk_time':d_walk_time_total,
           'Other_time':d_other_time_total,
           'Traffic_time':d_traffic_time,
           'Waiting_time':d_waiting_time,
           'Total_time':d_total_time,
           'Transfer':d_transfer}
    d_final=DataFrame(d_dic)
        
    print(d_final)
        '''

'\nfor o in range (len(farm[\'lat\'])):\n    \n    d_metro_time_total=[]\n    d_walk_time_total=[]\n    d_other_time_total=[]\n    d_total_time=[]\n    d_waiting_time=[]\n    d_status=[]\n    d_transfer=[]\n    d_traffic_time=[]\n    \n    for d in range(len(shop[\'lat\'])):\n        s=stod(str(farm[\'lat\'][o]),str(farm[\'lng\'][o]),str(shop[\'lat\'][d]),str(shop[\'lng\'][d]))\n        \n        if s.find(\'乘地铁\')==-1:\n            if s.find(\'乘\')!=-1:\n                result=\'Only bus can arrive\'\n                i_walk_time_total=-10000\n                i_other_time_total=10000\n                \n            elif s.find(\'乘\')==-1 and s.find(\'步行\')!=-1:\n                result=\'Within walking distance\'\n                i_walk_time_total=10000\n                i_other_time_total=-10000\n                \n            else:\n                result=\'Wrong cause without walking data, bus data or metro data\'\n                i_walk_time_total=0\n                i_other_time_total=

In [6]:
only_bus=[]
only_walk=[]
wrong_data=[]
metro=[]
walk=[]
other=[]
traffic=[]
waiting=[]
total=[]
transfer=[]

for o in range (len(farm['lat'])):
    
    d_metro_time_total=[]
    d_walk_time_total=[]
    d_other_time_total=[]
    d_total_time=[]
    d_waiting_time=[]
    d_status=[]
    d_transfer=[]
    d_traffic_time=[]
    
    for d in range(len(shop['lat'])):
        s=str(stod(str(farm['lat'][o]),str(farm['lng'][o]),str(shop['lat'][d]),str(shop['lng'][d])))
        
        if s.find('乘地铁')==-1:
            if s.find('乘')!=-1:
                result='Only bus can arrive'
                i_walk_time_total=-10000
                i_other_time_total=10000
                
            elif s.find('乘')==-1 and s.find('步行')!=-1:
                result='Within walking distance'
                i_walk_time_total=10000
                i_other_time_total=-10000
                
            else:
                result='Wrong cause without walking data, bus data or metro data'
                i_walk_time_total=0
                i_other_time_total=0
            i_metro_time_total=0
            i_total_time=0
            i_waiting_time=0
            i_transfer=-10000
            i_traffic_time=0
        
        elif s.find('乘地铁')!=-1:
            result='Can be arrived by metro'
            i_transfer=s.count('站内换乘')
            first_split = s.split("/")
            dic= {'step':first_split}
            table = DataFrame(dic)
            table[['step','time']] = table['step'].str.split(':', expand=True)
            # table['time'] = table['time'].fillna(0)
    
            i_metro_time=[]
            i_walk_time=[]
            i_other_time=[]
            for i in range(len(table)):
                if table['step'][i].find('乘地铁')!=-1 or table['step'][i].find('站内换乘')!=-1:
                    i_metro_time.append(float(table['time'][i]))
                elif table['step'][i].find('乘地铁')==-1 and table['step'][i].find('步行')!=-1 and table['step'][i].find('站内换乘')==-1:
                    i_walk_time.append(float(table['time'][i]))
                elif table['step'][i].find('Total time')!=-1:
                    i_total_time=float(table['time'][i])
                else:
                    i_other_time.append(float(table['time'][i]))
    
            i_metro_time_total=sum(i_metro_time)
            i_walk_time_total=sum(i_walk_time)
            i_other_time_total=sum(i_other_time)
            i_traffic_time=i_metro_time_total+i_walk_time_total+i_other_time_total
            i_waiting_time=i_total_time-i_traffic_time
    
        else:
            result='Wrong cause contradict result'
            i_metro_time_total=404
            i_walk_time_total=404
            i_other_time_total=404
            i_total_time=404
            i_waiting_time=404
            i_transfer=404
            i_traffic_time=404
        
        d_metro_time_total.append(i_metro_time_total)
        d_walk_time_total.append(i_walk_time_total)
        d_other_time_total.append(i_other_time_total)
        d_total_time.append(i_total_time)
        d_waiting_time.append(i_waiting_time)
        d_status.append(result)
        d_transfer.append(i_transfer)
        d_traffic_time.append(i_traffic_time)
        
    d_dic={'Destination':shop['name'],
           'Status':d_status,
           'Metro_time':d_metro_time_total,
           'Walk_time':d_walk_time_total,
           'Other_time':d_other_time_total,
           'Traffic_time':d_traffic_time,
           'Waiting_time':d_waiting_time,
           'Total_time':d_total_time,
           'Transfer':d_transfer}
    d_final=DataFrame(d_dic)
    
    f_only_bus=[]
    f_only_walk=[]
    f_wrong_data=[]
    f_metro=[]
    f_walk=[]
    f_other=[]
    f_traffic=[]
    f_waiting=[]
    f_total=[]
    f_transfer=[]
    for f in range (len(d_final['Destination'])):
        if d_final['Status'][f]=='Only bus can arrive':
            f_only_bus.append(d_final['Other_time'][f])
        elif d_final['Status'][f]=='Within walking distance':
            f_only_walk.append(d_final['Walk_time'][f])
        elif d_final['Status'][f]=='Can be arrived by metro':
            f_metro.append(d_final['Metro_time'][f])
            f_walk.append(d_final['Walk_time'][f])
            f_other.append(d_final['Other_time'][f])
            f_traffic.append(d_final['Traffic_time'][f])
            f_waiting.append(d_final['Waiting_time'][f])
            f_total.append(d_final['Total_time'][f])
            f_transfer.append(d_final['Transfer'][f])
        else:
            f_wrong_data.append(d_final['Transfer'][f])
    
    only_bus.append(len(f_only_bus))
    only_walk.append(len(f_only_walk))
    wrong_data.append(len(f_wrong_data))
    metro.append(ave(f_metro))
    walk.append(ave(f_walk))
    other.append(ave(f_other))
    traffic.append(ave(f_traffic))
    waiting.append(ave(f_waiting))
    total.append(ave(f_total))
    transfer.append(ave(f_transfer))
    print(str(only_bus)+","+str(only_walk)+","+str(wrong_data)+","+str(metro)+","+str(walk)+","+str(other)+","+str(traffic)+","+str(waiting)+","+str(total)+","+str(transfer))

dic={'Origin':farm['name'],
     'Average_metro_time':metro,
     'Average_walk_time':walk,
     'Average_other_time':other,
     'Average_traffic_time':traffic,
     'Average_waiting_time':waiting,
     'Average_total_time':total,
     'Average_metro_line_transfer':transfer,
    'Count_of_only_bus_round':only_bus,
    'Count_of only_walk_round':only_walk,
    'Count_of_none_data':wrong_data}
final=DataFrame(dic)
final

[0],[0],[3],[43.43504273504275],[30.375213675213683],[60.54423076923076],[134.35448717948714],[22.403846153846153],[156.75833333333333],[0.5769230769230769]
[0, 0],[0, 0],[3, 0],[43.43504273504275, 57.99197530864197],[30.375213675213683, 25.46440329218106],[60.54423076923076, 24.159670781893006],[134.35448717948714, 107.61604938271603],[22.403846153846153, 20.21604938271605],[156.75833333333333, 127.83209876543208],[0.5769230769230769, 1.308641975308642]
[0, 0, 0],[0, 0, 0],[3, 0, 1],[43.43504273504275, 57.99197530864197, 38.363749999999996],[30.375213675213683, 25.46440329218106, 38.30479166666665],[60.54423076923076, 24.159670781893006, 41.179374999999986],[134.35448717948714, 107.61604938271603, 117.84791666666668],[22.403846153846153, 20.21604938271605, 15.71875],[156.75833333333333, 127.83209876543208, 133.56666666666663],[0.5769230769230769, 1.308641975308642, 0.325]
[0, 0, 0, 0],[0, 0, 0, 0],[3, 0, 1, 4],[43.43504273504275, 57.99197530864197, 38.363749999999996, 41.1062770562770

,Origin,Average_metro_time,Average_walk_time,Average_other_time,Average_traffic_time,Average_waiting_time,Average_total_time,Average_metro_line_transfer,Count_of_only_bus_round,Count_of only_walk_round,Count_of_none_data
0,鹏飞路310号,43.435043,30.375214,60.544231,134.354487,22.403846,156.758333,0.576923,0,0,3
1,双晖现代农业,57.991975,25.464403,24.159671,107.616049,20.216049,127.832099,1.308642,0,0,0
2,宝安区福永街道办事处凤凰社区,38.363750,38.304792,41.179375,117.847917,15.718750,133.566667,0.325000,0,0,1
3,同乐社区,41.106277,28.792208,40.616883,110.515368,15.616883,126.132251,0.415584,0,0,4
4,新羌社区,47.479218,40.452263,46.871399,134.802881,18.425926,153.228807,0.691358,0,0,0
5,凤歧路20号,38.346042,34.714792,16.955000,90.015833,16.375000,106.390833,0.650000,0,0,1
6,大鹏街道,36.441026,24.757906,58.487179,119.686111,18.653419,138.339530,0.461538,0,0,3
7,新大路,28.825316,31.879747,86.459916,147.164979,18.891561,166.056540,0.392405,0,0,2
8,华润五丰农产品深圳有限公司,45.141558,33.119481,11.265152,89.526190,11.201299,100.727489,0.935065,0,0,4
9,灿城农产品物流园,41.390741,25.840123,34.738477,101.969342,14.043210,116.012551,0.419753,0,0,0


In [7]:
final=DataFrame(dic)
final

,Origin,Average_metro_time,Average_walk_time,Average_other_time,Average_traffic_time,Average_waiting_time,Average_total_time,Average_metro_line_transfer,Count_of_only_bus_round,Count_of only_walk_round,Count_of_none_data
0,鹏飞路310号,43.435043,30.375214,60.544231,134.354487,22.403846,156.758333,0.576923,0,0,3
1,双晖现代农业,57.991975,25.464403,24.159671,107.616049,20.216049,127.832099,1.308642,0,0,0
2,宝安区福永街道办事处凤凰社区,38.363750,38.304792,41.179375,117.847917,15.718750,133.566667,0.325000,0,0,1
3,同乐社区,41.106277,28.792208,40.616883,110.515368,15.616883,126.132251,0.415584,0,0,4
4,新羌社区,47.479218,40.452263,46.871399,134.802881,18.425926,153.228807,0.691358,0,0,0
5,凤歧路20号,38.346042,34.714792,16.955000,90.015833,16.375000,106.390833,0.650000,0,0,1
6,大鹏街道,36.441026,24.757906,58.487179,119.686111,18.653419,138.339530,0.461538,0,0,3
7,新大路,28.825316,31.879747,86.459916,147.164979,18.891561,166.056540,0.392405,0,0,2
8,华润五丰农产品深圳有限公司,45.141558,33.119481,11.265152,89.526190,11.201299,100.727489,0.935065,0,0,4
9,灿城农产品物流园,41.390741,25.840123,34.738477,101.969342,14.043210,116.012551,0.419753,0,0,0


In [8]:
final.to_csv('output_data/farm_to_shop.csv')

reference

https://www.cnblogs.com/jqpy1994/p/10842795.html

https://www.pythonf.cn/read/45820